## This notebook is to calculate metrics for data quality assessment of time interval maps.

1. Importing necessary columns from Excel
2. Declaring categories, strata, time intervals
3. Creating a dictionary for unique combination of entries
4. Converting the sample contingency table entries into a table for entire population using sampling intensity.
5. Calculating Commission and Omission entries for each category for all time intervals
6. Exporting the table as JSON

In [ ]:
pip install openpyxl

#### Import necessary python libraries

In [ ]:
import pandas as pd
from pprint import pprint
import json

#### Read PIE Error Excel data

In [ ]:
file_path = './input_data/PIE_Error_Vishal_Edited.xlsx'  # Replace with your file path
sheet_name = 'Data'  # Replace with the name of the sheet you want to read
columns_names = ['Stratum', 'Interval', 'MapInterval', 'RefInterval'] # specific columns to read
df = pd.read_excel(file_path, sheet_name=sheet_name, usecols= columns_names)

In [ ]:
df.head()

#### Unique values in columns used to create dictionary keys

In [ ]:
intervals = ['2010-2012', '2012-2014', '2014-2016', '2016-2018', '2018-2021']
categories = ['Stable Presence', 'Loss', 'Gain', 'Stable Absence']
stratum = ['All Absence', 'Gain', 'Other', 'Loss', 'All Presence']

In [ ]:
number_of_time_intervals = len(intervals)
number_of_categories = len(categories)
number_of_strata = len(stratum)

#### Creating a dictionary for unique combination of entries

In [ ]:
dic = {}
for strata in stratum:
    for category in categories:
        for category2 in categories:
            label = strata + '_' + category + '_' + category2
            dic[label] = 0   

#### Looping through the rows to make the count for each entry 

In [ ]:
sample_entries_all_intervals = {}
for interval in intervals:
    dic = {}
    for strata in stratum:
        for category in categories:
            for category2 in categories:
                label = strata + '_' + category + '_' + category2
                dic[label] = 0   
    
    df2 = df[df['Interval'] == interval]
    for i in range(len(df2)):
        strata = columns_names[0]
        map_label = columns_names[2]
        ref_label = columns_names[3]
        
        a = df2[strata].iloc[i]
        b = df2[map_label].iloc[i]
        c = df2[ref_label].iloc[i]
        
        dic[f'{a}_{b}_{c}'] += 1
    sample_entries_all_intervals[interval] = dic

In [ ]:
pprint(sample_entries_all_intervals)

#### Weights calculated for each strata depending on the sampling intensity

In [ ]:
# in pixels
weights = {
           'All Absence': 1176064.92,
           'All Presence' : 81422.60,
           'Gain': 27678.11,
           'Loss': 37697.48,
           'Other': 83916.21
}
# in acres
weights2 = {
           'All Absence': 70.56,
           'All Presence' : 4.88,
           'Gain': 1.66,
           'Loss': 2.26,
           'Other': 5.03
}

#### Converting the sample contingency table entries into an estimated population table

Using the formula #5.2 from Chapter 5 "Application to Categorical Error Assessment with Sampling"
of the book "Metrics That Make a Difference" https://link.springer.com/book/10.1007/978-3-030-70765-1

In [ ]:
population_entries_all_intervals = {}
for interval in intervals:   
    dic = {}
    for category in categories:
        for category2 in categories:
            label = category + '_' + category2
            dic[label] = 0
    
    for entry in dic:
        for index, key in enumerate(weights.keys()):
            entry2 = key + '_' + entry
            dic[entry] += weights2[key]*sample_entries_all_intervals[interval][entry2]
    population_entries_all_intervals[interval] = dic

In [ ]:
population_entries_all_intervals

#### Entries for each category combined together for all intervals

In [ ]:
population_entries_redefined =  {}
for category in categories:
    for category2 in categories:
        label = category + '_' + category2
        population_entries_redefined[label] = []

# Iterate over each year's data
for year_data in population_entries_all_intervals.keys():        
    # Iterate over each category
    for category in population_entries_redefined.keys():
        # Extract count for the current category from the current year's data
        count = population_entries_all_intervals[year_data].get(category, 0)
        # Append the count to the corresponding array
        population_entries_redefined[category].append(count)


In [ ]:
population_entries_redefined

#### Calculating Commission for each category for all time intervals

In [ ]:
commissions = {}
for cat in range(number_of_categories):
    # filter entries that have reference categories different than a given map category (commission) 
    commission_entries =  {key: value for key, value in population_entries_redefined.items() if key.startswith(f'{categories[cat]}_') and key != (f'{categories[cat]}_{categories[cat]}')}
    commission_all_intervals = [0]*number_of_time_intervals
    for j in range(number_of_time_intervals):
        #print(commission_entries)
        for entry in commission_entries:
            #print(population_entries_all_intervals[intervals[j]][entry])
            commission_all_intervals[j] += population_entries_all_intervals[intervals[j]][entry]
        #print(commission_all_intervals[j], categories[cat])
    commissions[f'{categories[cat]}_Commission'] =  commission_all_intervals

#### Add commissions to the dictionary

In [ ]:
population_entries_redefined.update(commissions)

In [ ]:
pprint(population_entries_redefined)

#### Calculating Omission for each category for all time intervals

In [ ]:
omissions = {}
for cat in range(number_of_categories):
    # filter entries that have map categories different than a given reference category (omission)
    omission_entries =  {key: value for key, value in population_entries_redefined.items() if key.endswith(f'_{categories[cat]}') and key != (f'{categories[cat]}_{categories[cat]}')}
    omission_all_intervals = [0]*number_of_time_intervals
    for j in range(number_of_time_intervals):
        #print(omission_entries)
        for entry in omission_entries:
            #print(population_entries_all_intervals[intervals[j]][entry])
            omission_all_intervals[j] += population_entries_all_intervals[intervals[j]][entry]
        #print(omission_all_intervals[j], categories[cat])
    omissions[f'{categories[cat]}_Omission'] =  omission_all_intervals

#### Add omissions to the dictionary

In [ ]:
population_entries_redefined.update(omissions)

In [ ]:
pprint(population_entries_redefined)

#### Rounding data in each entry to two decimals

In [ ]:
def convert_decimals_to_2(data):
    if isinstance(data, dict):
        return {key: convert_decimals_to_2(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_decimals_to_2(item) for item in data]
    elif isinstance(data, (int, float)):
        return round(data, 2)
    else:
        return data

In [ ]:
result_dict = convert_decimals_to_2(population_entries_redefined)

In [ ]:
pprint(result_dict)

##### Exporting the table as JSON

In [ ]:
# Specify the file path
json_file_path = './output_data/table2.json'

# Write the data to the JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(result_dict, json_file, indent=4)